### Let's implement toy pp example

In [ ]:
import random

import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# set seed for reproducibility

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
# We will use FashionMNIST dataset

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 32)
test_dataloader = DataLoader(test_data, batch_size = 32)

train_dataloader.dataset[0][0].shape

In [ ]:
class SingleNeuralNetwork(nn.Module):
    def __init__(self):
        super(SingleNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
my_single_net = SingleNeuralNetwork()
my_single_net

In [ ]:
def single_train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()

        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward(retain_graph = True)
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"train loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

def single_test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X , y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()

# set hyperparams
learning_rate = 1e-3
epochs = 5

single_optimizer = torch.optim.SGD(my_single_net.parameters(), lr = learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------")
    single_train_loop(train_dataloader, my_single_net, loss_fn, single_optimizer)
    single_test_loop(test_dataloader, my_single_net, loss_fn)
print("Done!")

In [ ]:
class NeuralNetwork1(nn.Module):
    def __init__(self):
        super(NeuralNetwork1, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class NeuralNetwork2(nn.Module):
    def __init__(self):
        super(NeuralNetwork2, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model1 = NeuralNetwork1()
model2 = NeuralNetwork2()

In [ ]:
optimizer1 = torch.optim.SGD(model1.parameters(), lr = learning_rate)
optimizer2 = torch.optim.SGD(model2.parameters(), lr = learning_rate)

Let's Complete pipeline parallel train loop

In [ ]:
def pp_train_loop(dataloader, model1, model2, loss_fn, optimizer1, optimizer2):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred1 = model1(X)

        # input of model2 forward is output of model1 forward!
        # Your codes start here
        pred2 = 
        # Your codes end here

        # loss can be obtained with output of model2
        # Your codes start here
        loss2 = 
        # Your codes end here

        optimizer2.zero_grad()
        pred1.retain_grad()
        loss2.backward(retain_graph = True)
        inter_grad = pred1.grad
        optimizer2.step()

        optimizer1.zero_grad()

        # input of model1 backward is output of model2 backward!
        # Your codes start here
        pred1_grad = 
        # Your codes end here
        pred1.backward(pred1_grad)
        optimizer1.step()

        if batch % 1000 == 0:
            loss2, current = loss2.item(), batch * len(X)
            print(f"loss : {loss2:>7f} [{current:>5d}/{size:>5d}]")

def pp_test_loop(dataloader, model1, model2, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X , y in dataloader:
            pred1 = model1(X)
            pred2 = model2(pred1)
            test_loss += loss_fn(pred2, y).item()
            correct += (pred2.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------")
    pp_train_loop(train_dataloader, model1, model2, loss_fn, optimizer1, optimizer2)
    pp_test_loop(test_dataloader, model1, model2, loss_fn)
print("Done!")

In [ ]:
single_net_param_sum = 0
for param in my_single_net.parameters():
    flattened_param = param.reshape(1,-1)
    # Your codes start here
    ? += len(flattened_param[0])
    # Your codes end here
single_net_param_sum

In [ ]:
partitioned_net_param_sum = 0
for param in model1.parameters():
    flattened_param = param.reshape(1,-1)
    # Your codes start here
    ? += len(flattened_param[0])
    # Your codes end here
for param in model2.parameters():
    flattened_param = param.reshape(1,-1)
    # Your codes start here
    ? += len(flattened_param[0])
    # Your codes end here
partitioned_net_param_sum